## 用語

* 活性化関数
  * 複数のニューロンからあるニューロンへの入力は、全結合などの線形変換で1つにまとめられる。活性化関数は、その数値を次のニューロンに「どのように出力するか」、つまり「どう活性化するか」が定義されたものだ。
    * 活性化関数は数式自体よりどのような形状のグラフになっているかが重要
  * この活性化関数による返還んは`非線形変換`である必要がある。
    * なぜなら、何層もニューロンを重ねていったときに線形だと多層にする意味がなくなるため。
  * 活性化関数の例
    * ステップ関数、シグモイド関数、LeRU関数など
  * ただし、出力層に関しては次の層がないため次のような関数が使われる。
    * 恒等関数
      * 入力をそのまま出力する。
      * 回帰問題で使われる
    * ソフトマックス関数
      * ニューラルネットの出力を0～1の範囲に正規化する。つまり確率と解釈できる。
      * 分類問題で使われる

In [1]:
import sys, os

sys.path.append(os.pardir)
from dataset.mnist import load_mnist
import numpy as np
from PIL import Image
import pickle
from common.functions import sigmoid, softmax

In [2]:

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()


# normalize:入力画像を0~1に正規化する。
# flatten:画像を1次元配列に変換する
# noe_hot_label:ラベルをone hot表現（0か1)にする
#(訓練画像、訓練ラベル),(テスト画像,テストラベル)を出力する
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label)

# 1次元の配列の次元数が表示
print(img.shape)
# 2次元に再変換
img = img.reshape(28, 28)
# 2次元配列の次元数が表示
print(img.shape)
img_show(img)


5
(784,)
(28, 28)


In [3]:



def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(
        normalize=True, flatten=True, one_hot_label=False
    )
    # テスト画像とラベルだけを返す
    return x_test, t_test


def init_network():
    """学習済みネットワークを取得

    Returns:
        _type_: _description_
    """
    with open("./ch03/sample_weight.pkl", "rb") as f:
        network = pickle.load(f)
    print(f'W1:{network["W1"].shape}, W2:{network["W2"].shape}, W3:{network["W3"].shape}')
    return network


def predict(network, x):
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]
    # 1層目の処理
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)# 活性化関数
    # 2層目の処理
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)# 活性化関数
    # 3層目（出力層）の処理
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)# 分類問題に使うためソフトマックス関数を使う。

    return y

# テスト画像とラベルを取得
x, t = get_data()
# xには28×28の画像が10000枚格納されている。
# 実際には画像は1次元に変換されるので(10000, 784)というサイズになっている

# 学習済みのパラメータを取得
network = init_network()
batch_size = 100
accuracy_cnt = 0
# 分類データが正確に分類できるかを推論する
for i in range(0,len(x),batch_size):
    # iからi+batch_sizeのデータを取得
    x_batch = x[i:i+batch_size]
    #print(x_batch.shape)
    # テスト画像を学習モデルに適用し、分類させる。
    y_batch = predict(network, x_batch)
    # yリストの中から、最大の確率を取得
    # axis=1とすることで、多次元配列の1次元目の中から最大の物を抽出
    p = np.argmax(y_batch,axis=1)
    # 最大確率となるインデックスがラベルデータと一致するか判定
    #　p==t[i:i+batch_size]とすると、pに一致する部分がTrue,不一致がFalseとなる。
    # このようなboolの配列が返却され、そのsumを取るとTrueの個数として取得できる。
    accuracy_cnt+=np.sum(p==t[i:i+batch_size])

#「分類が成功した数/入力データのサイズ」で精度を算出
print("Accuracy:" + str(float(accuracy_cnt) / len(x)))


W1:(784, 50), W2:(50, 100), W3:(100, 10)
Accuracy:0.9352
